In [1]:
# ANOVA- Analysis of variance
# Grup ortalamalarını kıyaslayacağız.

In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp,shapiro,levene,ttest_ind, \
    mannwhitneyu,pearsonr,spearmanr,kendalltau,f_oneway,kruskal
from statsmodels.stats.proportion import proportions_ztest

In [3]:
df = sns.load_dataset("tips")

In [4]:
# Günlere göre toplam hesap ortalamaları
df.groupby("day")["total_bill"].mean()

day
Thur    17.682742
Fri     17.151579
Sat     20.441379
Sun     21.410000
Name: total_bill, dtype: float64

In [ ]:
# H0: m1=m2=m3=m4
# Grup ortalamaları arasında fark yoktur.

# H1: Fark vardır

# Varsayım sağlanıyorsa -> one way anova
# Varsayım sağlanmıyorsa -> kruskal

In [5]:
# H0: Normal dağılım varsayımı sağlanmaktadır
for group in (df["day"].unique()):
    pvalue = shapiro(df.loc[df["day"] == group, "total_bill"])[1]
    print(group, "p-value= %.4f" % pvalue)
    
# H0 sağlanmaz.    
# pvalue < 0.05

Sun p-value= 0.0036
Sat p-value= 0.0000
Thur p-value= 0.0000
Fri p-value= 0.0409


In [6]:
# H0: Varyans homojenliği varsayımı sağlanmaktadır.
test_stat, pvalue = levene(df.loc[df["day"] == "Sun","total_bill"],
                           df.loc[df["day"] == "Sat","total_bill"],
                           df.loc[df["day"] == "Thur","total_bill"],
                           df.loc[df["day"] == "Fri","total_bill"])

print("Test stat = %.4f, p-value= %.4f" % (test_stat, pvalue))

# H0 reddedilemez.
# pvalue > 0.05

Test stat = 0.6654, p-value= 0.5741


In [7]:
# Varsayım sağlanmadı ama biz sağlandı varsayalım
f_oneway(df.loc[df["day"] == "Sun","total_bill"],
         df.loc[df["day"] == "Sat","total_bill"],
         df.loc[df["day"] == "Thur","total_bill"],
         df.loc[df["day"] == "Fri","total_bill"])

# pvalue < 0.05
# H0 reddedilir.

# Anlamlı fark vardır.

F_onewayResult(statistic=2.7674794432863363, pvalue=0.04245383328952047)

In [9]:
# Varsayımlar sağlanmadı doğru kısım
kruskal(df.loc[df["day"] == "Sun","total_bill"],
         df.loc[df["day"] == "Sat","total_bill"],
         df.loc[df["day"] == "Thur","total_bill"],
         df.loc[df["day"] == "Fri","total_bill"])

# pvalue < 0.05
# H0 reddedilir.

# Anlamlı fark vardır.

KruskalResult(statistic=10.403076391436972, pvalue=0.015433008201042065)

In [10]:
# Fark var ama hangisinde, hangi grupta
from statsmodels.stats.multicomp import MultiComparison

In [12]:
comparison = MultiComparison(df["total_bill"],df["day"])
tukey = comparison.tukeyhsd(0.05)
print(tukey.summary())

# Hepsinin p-value > 0.05 -> p-adj
# Reddedilmez -> Fark yoktur.
# Tüm gruba göre fark var, tek tek baktık fark yok.
# Perşembe-Pazar çok yakın farklılık burada olabilir

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   Fri    Sat   3.2898 0.4541 -2.4799  9.0595  False
   Fri    Sun   4.2584 0.2371 -1.5856 10.1025  False
   Fri   Thur   0.5312 0.9957 -5.4434  6.5057  False
   Sat    Sun   0.9686 0.8968 -2.6088   4.546  False
   Sat   Thur  -2.7586 0.2374 -6.5455  1.0282  False
   Sun   Thur  -3.7273 0.0668 -7.6264  0.1719  False
----------------------------------------------------
